# Data Cleaning with Python

## Introduction
In this notebook, we will share a few common tasks you will perform in data cleaning, using the **[New York City Airbnb](https://github.com/ManarOmar/New-York-Airbnb-2019)** Open Data from Kaggle as an example. 

In particular, it will be centered around these 3 topics below:

* Missing data
* Inconsistent data/Irrelevant features
* Outliers 

## 1. Overview of the data

### Importing Libraries

First we start by importing the necessary libraries for data cleaning.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

%matplotlib inline

### Load the data

Then we load the data. In this case, we loaded it from a csv file hosted on Github, but you can upload the csv file and import that data using pd.read_csv()
Notice that we copy the original dataset using .copy(). This is for a data cleaning example later on in the article. It’s also good practice to copy the data set when you want to test something out so you won’t have to re-run the entire notebook if you make a mistake somewhere.


In [ ]:
airbnb_url = 'https://raw.githubusercontent.com/ManarOmar/New-York-Airbnb-2019/master/AB_NYC_2019.csv'

airbnb_ori = pd.read_csv(airbnb_url)
airbnb = airbnb_ori.copy()
airbnb.head()

### Data info
Calling info() on our dataset tells us tons of information about our data frame like the shape (rows, columns), the data type of our features, and the memory usage.

In [ ]:
airbnb.info()

### Data Type
Separating our features into numerical and categorical early on is useful and here is a helper function that does that. This is accomplished by using the select_dtypes() function that columns with the ‘object’ data type as categorical and the rest as numerical.

In [ ]:
cat_df = airbnb.select_dtypes(include=['object'])
num_df = airbnb.select_dtypes(exclude=['object'])

def printColumnTypes(non_numeric_df, numeric_df):
    '''separates non-numeric and numeric columns'''
    print("Non-Numeric columns:")
    for col in non_numeric_df:
        print(f"{col}")
    print("")
    print("Numeric columns:")
    for col in numeric_df:
        print(f"{col}")
        
printColumnTypes(cat_df, num_df)

Now we have some idea of our data, we can move on to cleaning it by first checking for missing values.

## 2. Visualize Missing Data

Missing data is common in all kinds of data and is tricky to deal with. Most machine learning techniques do not work with missing values and it has to be addressed early on.

The two common methods to deal with missing values are

* dropping the rows / columns
* imputing them based on other observations i.e. the mean or median

There are a few problems to these approaches.

For example, by dropping rows/columns, you’re essentially losing information that might be useful for prediction
On the other hand, imputing values will introduce bias to your data but it still might better than removing your features.

Here are a couple useful helper functions you can use to visualize missing values.

### Columns with missing data

This function only prints out columns with missing values, and shows its amount

If you want to see missing values for all columns, use this command:

*df.isnull().sum()*

In [ ]:
def missing_cols(df):
    '''prints out columns with its amount of missing values'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        total += missing_vals
        if missing_vals != 0:
            print(f"{col} => {df[col].isnull().sum()}")
    
    if total == 0:
        print("no missing values left")
            
missing_cols(airbnb)

### Percentage missing

This gives you the percentage of missing values in each of the columns. Knowing the percentage can be useful in determining whether you should drop the column.

The percentage is calculated using the **mean**

In [ ]:
def perc_missing(df):
    '''prints out columns with missing values with its %'''
    for col in df.columns:
        pct = df[col].isna().mean() * 100
        if (pct != 0):
            print('{} => {}%'.format(col, round(pct, 2)))
    
perc_missing(airbnb)

### Heatmap of missing values
Heatmaps are also useful to visualize your missing values, in particular at which point of the data do missing values exists.

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(airbnb.isnull(), yticklabels=False, cmap='viridis', cbar=False)

Now that you know which columns have missing values, it’s time to deal with them.


## 3. Dealing with Missing Data

Some common techniques shared in this lab are:
1. Drop the feature
2. Drop the row
3. Impute the missing value
4. Replace it


### Dropping feature
Dropping feature usually isn’t recommended because you’re losing information. But if you’re sure that the column isn’t important, or simply has too many missing values, you can choose to drop them. For example, for this dataset, the host_name column was removed for ethical reasons, and id was removed because it was was unnecessary.

To drop features, use drop and set axis to 1 and inplace to true. Axis is 1 because we want to drop columns (0 means row), and inplace is True because you're transforming it directly on your dataset.

In [ ]:
# Drop unnecessary columns that are not important
colsToDrop = ['id','host_name','last_review']

airbnb.drop(colsToDrop, axis=1, inplace=True)

missing_cols(airbnb)

### Dropping the row

In [ ]:
# remove rows with missing values in price
airbnb['price'].dropna(inplace=True)

### Imputing

For imputing, there are 3 main techniques shown below.
1. **fillna** — filling in null values based on given value (mean, median, mode, or specified value)
2. **bfill** / ffill — stands for backward fill and forward fill (filling in missing values based on the value after or before the column.)
3. Simple Imputer — Sk-learn’s built-in function that imputes missing values (commonly used alongside a pipeline when building ML models)

Below you can find examples of applying these methods to the price column if it had missing values.

In [ ]:
# imputing price with mean
price_mean_value = round(airbnb['price'].mean(), 2)
airbnb['price'].fillna(price_mean_value, inplace=True)

# imputing price with median
price_median_value = round(airbnb['price'].median(), 2)
airbnb['price'].fillna(price_median_value, inplace=True)

# imputing with bfill or ffill
airbnb['price'].bfill(inplace=True)
airbnb['price'].ffill(inplace=True)

# imputing with SimpleImputor from the sklearn library
from sklearn.impute import SimpleImputer
# define the imputer
imr = SimpleImputer(missing_values=np.nan, strategy='mean') # or median

airbnb[['price']] = imr.fit_transform(airbnb[['price']])

# use strategy = 'most_frequent' for categorical data

### Replace

To replace values, the ***fillna*** function is also used.

You define the value you want to replace in the key, and the substitute in the value — {column_name: replacement_for_NA}

Here are examples for replacing values in the columns reviews_per_month and name

In [ ]:
# replace null values in reviews_per_month with 0 
airbnb.fillna({'reviews_per_month':0}, inplace=True)

missing_cols(airbnb)

In [ ]:
# replace null values in name with 'None'
airbnb.fillna({'name':'None'}, inplace=True)

missing_cols(airbnb)

Now we have no missing values left!

Let’s move on to dealing with inconsistent or irrelevant features.

## 4. Inconsistent data/Irrelevant features

Inconsistent data refers to things like spelling errors in your data, column names that are not relevant to the data, the wrong data type, etc.

Here are a couple examples for dealing with these issues.


### Remove rows based on regex

Let’s say you want to remove rows that contain a certain word. 

Take the word noisy/Noisy as an example, and the function str.contains() is used to find the indexes that contain those rows.

Then, using the drop function and setting axis to index to drop those rows.

Printing out the number of rows, you can see it reduced by three.

In [ ]:
# example: remove rows that contain the target word
target = '[Nn]oisy'

noisy_airbnb = airbnb[airbnb['name'].str.contains(target, regex=True)]

# show rows that contains the word noisy
print(noisy_airbnb['name'])

# get the index that contains the word noisy
index_to_drop = noisy_airbnb['name'].index

# print(index_to_drop)

In [ ]:
# drop rows based on index
airbnb.drop(index_to_drop, axis='index', inplace=True)

print(len(airbnb_ori))
print(len(airbnb))

### Spelling errors in categorical data
Sometimes your categorical data might have spelling errors or different capitalization that can mess up your categorization.

I will be using the *neighbourhood_group* column as an example.

In [ ]:
airbnb['neighbourhood_group'].value_counts()

You can see the different types of neighborhoods are already well categorized. But what if it wasn’t?

To simulate a scenario where some of the data had capitalization or spelling issues, I sampled 2 rows from the data, and replaced them with the wrong spelling.

You can see now how the categorization is messed up. “Manhattan” and “manhatann” refer to the same thing, but they aren’t in the same category because of capitalization. Same goes for “brookln” due to spelling issues.

We can fix this by using the replace function in pandas. We first give the values that are wrong, then supply the right ones. Notice the values have to match each other in the list, i.e. “manhatann” → “Manhattan”.

In [ ]:
random_index = airbnb.sample(2, random_state = 10).index

# airbnb['neighbourhood_group'].loc[random_index]
## we randomly selected Manhattan and Brooklyn

wrong_spelling = ['manhatann', 'brookln']

# replace them with the wrong spelling
airbnb.loc[random_index,'neighbourhood_group'] = wrong_spelling
airbnb['neighbourhood_group'].value_counts()

In [ ]:
airbnb['neighbourhood_group'].replace(['manhatann', 'brookln'],
                             ['Manhattan', 'Brooklyn'], inplace=True)
airbnb['neighbourhood_group'].value_counts()

### Converting to DateTime

If you have data that should be a datetime object, but are strings, you can use the pd.to_datetime, and pass it the format that represents your data.

Just like that, the column has converted into a datatime data type.

In [ ]:
airbnb_ori['last_review'] = pd.to_datetime(airbnb_ori['last_review'], format='%Y-%m-%d')
airbnb_ori['last_review'].dtype.type

### Duplicates

There are cases where your rows have duplicate values, this could’ve happened due to some mishaps in your data collection.

To find out if you have duplicated values, call duplicated().any() on your data frame, and if it’s true, use the drop_duplicates function

You can also specify columns where you want to remove duplicate values like below.

In [ ]:
airbnb.duplicated().any()

## if true
# airbnb.drop_duplicates()

## if you want to drop duplicates at specific column
# airbnb.drop('col_name', axis=1, inplace=True).drop_duplicates()

### Change data type to reduce memory

Changing data type is common if you want to reduce memory usage.

To do so, you can use the astype(‘dtype’) function where you specify the dtype you want.

In this case, I changed the data type for the host_id column from int64 to int32

Observe the memory before changing the data type

In [ ]:
airbnb.info()

And after

In [ ]:
airbnb['host_id'] = airbnb['host_id'].astype('int32')
airbnb.info()

You can see the memory reduced from 6.5+ to 6.3+ MB.

[Here](https://stackoverflow.com/questions/15891038/change-column-type-in-pandas#:~:text=The%20best%20way%20to%20convert%20one%20or%20more%20columns%20of,floating%20point%20numbers%20as%20appropriate.) is more information on changing data types.

## 5. Outliers

Outliers can be dangerous as they can skew your model and give you predictions that are biased and erroneous.

The best way to find outliers is to use the describe function and look at information such as maximum and mean.

In [ ]:
airbnb['price'].describe()

You can also plot a histogram and look at the distribution of your data.

In this histogram, you can see that most of the data is around 0 to 5000.

In [ ]:
plt.figure(figsize=(10, 6))
airbnb['price'].hist(bins=100)

A boxplot is also useful in detecting outliers.

As you can see, the price column has multiple data points that are outliers (above of the maximum in the boxplot)

In [ ]:
plt.figure(figsize=(10, 6))
airbnb.boxplot(column=['price'])

For categorical data, you can plot a bar chart to see whether a particular category to view the count of the categories.

Outliers in categorical data is tricky, because you have to determine whether it’s appropriate to call it an outlier based on context.

In some cases, outliers depend on context. In my example, you see that Manhattan and Brooklyn has significantly more data than Staten Island. This doesn’t count as an outlier, since Manhattan and Brooklyn has a higher housing density as compared to Staten Island.

In [ ]:
plt.figure(figsize=(10, 6))
airbnb['neighbourhood_group'].value_counts().plot.bar()

### Dealing with outliers
Dealing with outliers is similar to removing missing values, the only difference is the way you find outliers.

#### **Reference**
Neo, Benedict. 2021. “Data Cleaning with Python.” Bitgrit Data Science Publication. May 21, 2021.